In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


#Bước 1
* Đọc file

In [3]:
import json
import pandas as pd

with open('/content/drive/MyDrive/Colab Python/Processed_merge_data.json', 'r', encoding='utf-8') as file:
    data = json.load(file)


* Tạo data frame

In [4]:
df = pd.DataFrame(data)


#Xử lý cột dữ liệu
* Điền giá trị null trong project_legal, category, và extensions bằng chuỗi rỗng

In [5]:
df['project_legal'] = df['project_legal'].apply(lambda x: x if x is not None else [])
df['category'] = df['category'].apply(lambda x: x if x is not None else [])
df['extensions'] = df['extensions'].apply(lambda x: x if x is not None else [])


#Bước 2
- Explode các danh sách thành các hàng riêng biệt

In [6]:
df_exploded = df.explode('category').explode('extensions').explode('project_legal')


* Định nghĩa các cột phân loại và số

In [7]:
categorical_cols = ['category', 'extensions', 'project_legal']
numerical_cols = ['area', 'population_density']


* Đặc trưng và nhãn

In [8]:
features = ['area', 'category', 'extensions', 'project_legal', 'population_density']
target = 'average_price/1m2'

* Tiền xử lý cho các cột phân loại

In [9]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])


* Tiền xử lý cho các cột số

In [10]:
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean'))
])


* Đưa các cột tiền xử lý vào ColumnTransformer

In [11]:
from sklearn.compose import ColumnTransformer

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])


* Chuẩn bị đặc trưng và nhãn

In [12]:
X = df_exploded[features]
y = df_exploded[target]


#Bước 3
* Tạo Pipeline cho mô hình

In [13]:
from sklearn.ensemble import RandomForestRegressor

pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor(n_estimators=100, random_state=42))
])


* Huấn luyện mô hình

In [14]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
pipeline.fit(X_train, y_train)


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer())]),
                                                  ['area',
                                                   'population_density']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(fill_value='missing',
                                                                                 strategy='constant')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['category', 'extensions',
                                                   'project_legal'])])),
                ('regressor', RandomForestRegressor(random_state=42))])

* Dự đoán trên tập kiểm tra

In [15]:
y_pred = pipeline.predict(X_test)

* Đánh giá mô hình

In [16]:
from sklearn.metrics import mean_squared_error

mse = mean_squared_error(y_test, y_pred)
rmse = mse ** 0.5

print(f'Root Mean Squared Error: {rmse}')


Root Mean Squared Error: 6093400.739607412


#Dự đoán giá nhà

In [17]:
def predict_price(new_data):
    # Tạo DataFrame từ dữ liệu đầu vào
    new_df = pd.DataFrame([new_data])

    # Xử lý các cột phân loại nếu giá trị là None
    new_df['project_legal'] = new_df['project_legal'].apply(lambda x: x if x is not None else [])
    new_df['category'] = new_df['category'].apply(lambda x: x if x is not None else [])
    new_df['extensions'] = new_df['extensions'].apply(lambda x: x if x is not None else [])

    # Chuyển đổi dữ liệu bằng pipeline
    new_df_exploded = new_df.explode('category').explode('extensions').explode('project_legal')
    return pipeline.predict(new_df_exploded)


In [18]:
# Ví dụ dự đoán với dữ liệu mới
new_data = {
        "area": 117.0,
        "category": [
            "căn hộ"
        ],
        "extensions": [
            "nội thất",
            "ban công",
            "an ninh"
        ],
        "project_legal": [
            "sổ đỏ"
        ],
        "population_density": 4149
    }

predicted_price_per_m2 = predict_price(new_data)
print(f'Predicted average price per m2: {predicted_price_per_m2[0]:,.2f}')

Predicted average price per m2: 53,673,290.60


In [19]:
new_data = {
        "area": 45.0,
        "category": [
            "nhà riêng"
        ],
        "extensions": None,
        "project_legal": [
            "sổ đỏ"
        ],
        "population_density": 29647
    }

predicted_price_per_m2 = predict_price(new_data)
print(f'Predicted average price per m2: {predicted_price_per_m2[0]:,.2f}')

Predicted average price per m2: 49,466,619.76


In [20]:
new_data = {
        "area": 36.0,
        "category": [
            "nhà riêng"
        ],
        "extensions": [
            "hỗ trợ vay",
            "trường học",
            "chợ"
        ],
        "project_legal": [
            "sổ đỏ"
        ],
        "population_density": 3757
    }

predicted_price_per_m2 = predict_price(new_data)
print(f'Predicted average price per m2: {predicted_price_per_m2[0]:,.2f}')

Predicted average price per m2: 73,464,616.43
